# アテンションレイヤー

アテンションは機械学習やAIにおいて、特にコンピュータビジョンで何年も前から存在する概念です{cite}`BALUJA1997329`。「ニューラルネットワーク」という言葉と同様に、アテンションは人間の脳が大量の視覚・聴覚入力を処理する際の注意機構に着想を得ています{cite}`BALUJA1997329`。**アテンションレイヤー**は、その注意機構を再現する深層学習レイヤーです。深層学習におけるアテンションについてはLuongら{cite}`luong2015effective`が詳細を解説しています。また、[こちら](http://d2l.ai/chapter_attention-mechanisms/index.html)で実践的な概要が紹介されています。アテンションレイヤーは言語のような系列のモデリングに非常に有用であることが経験的に示されており、現在では必要不可欠な存在となっています{cite}`vaswani2017attention`。アテンションレイヤーが最も良く使わているのは系列モデリングに用いる[**トランスフォーマー**](http://d2l.ai/chapter_attention-mechanisms/transformer.html)ニューラルネットワークです。また。グラフニューラルネットワークでもアテンションが使われることがあります。


```{margin}
アテンションには３つの入力があるため混乱しがちですが、これらの入力は実際は同一であることが多いことを後で説明します。
クエリーはキーのひとつであり、キーとバリューは等しいです。そして、クエリをまとめて入力すると、クエリー、キー、バリューが同一であることに気が付くでしょう。これがSelf-Attentionです。
```

```{admonition} Audience & Objectives
この章は、{doc}`layers` と {doc}`../math/tensors-and-shapes`を理解していることを前提に書かれているので、ブロードキャスト、行列やテンソルの形状には慣れていた方が良いでしょう。この章を終えるころには、以下の事ができるようになっているはずです。


  * アテンションレイヤーの形状や入出力の正しい指定
  * アテンションレイヤーの実装
  * 他のレイヤーにアテンションを適用する方法
```

アテンションレイヤーは、基本的には加重平均による次元削減です。これは単に各要素に何らかの方法で重み付けをし、その平均を計算しているだけです。これにより、アテンションは入力テンソルのランクを小さくします。ほとんどのレイヤーが１つもしくは２つの入力を取るのに対し、アテンションは3つの入力を取るという点で、珍しいレイヤーです。 これら3つの入力はそれぞれ、**クエリー**、**バリュー**、**キー**と呼ばれます。次元削減はバリューに対して行われます。つまり、バリューのランクが3であれば、出力のランクは2になります。クエリーはキーより１小さいランクであるべきです。キーとバリューは同じランクであるべきです。キーとクエリーは**アテンション機構** --方程式を意味する--に従ってバリューの重みを決定します。

下の表はこれら3つの入力をまとめたものです。多くの場合、クエリーはバッチ処理されるため、ランクが2になることに注意してください。入力クエリがバッチ処理されている場合、出力ランクも１ではなく同様に2になります。

|    |ランク| 形状 | 目的| 例|
|:----|-----|------|----|----:|
|クエリー| 1 | (アテンション特徴量の数) | キーに対してチェックを行う入力 | 特徴量ベクトルとして表現された1単語|
|キー| 2 | (系列の長さ, アテンション特徴量の数) | クエリーに対してアテンションを計算するために使用する | 特徴量ベクトルとして表現された文中の全単語|
|バリュー | 2 | (系列の長さ, アテンション特徴量の数) | 出力値を計算するために使用される | 文中の各単語に対応する数値ベクトル|
|出力 | 1 | (バリュー特徴量の数) | バリューのアテンションウェイトによる加重平均 | １つのベクトル |

## 例

アテンションは系列データで考えると分かりやすい概念です。「The sleepy child reads a book」という文章で考えてみましょう。文中の各単語はキーに相当し、単語を埋め込みで表現するとキーはランク2となります。例えば、「sleepy」という単語は、長さ3の埋め込みベクトル： $[2, 0, 1]$ で表現されるかもしれません。これらの埋め込みは標準的な言語の埋め込みから学習または取得されたものです。慣習的に、キーの0番目の軸は系列における位置を表し、1番目の軸がベクトルを表します。クエリーは多くの場合、「book」という単語のようにキーの中の一要素になっています。クエリーが文中のどの部分から影響を受けているかを見ているというのが、アテンションの重要なポイントです。「Book」は「child」と「reads」に強いアテンションを持つはずですが、「sleepy」ではそうはならないはずです。これを実際にベクトルとして計算したものをアテンションベクトル &\vec{b}& と呼ぶことはすぐにわかるでしょう。アテンションレイヤーの出力は、クエリーとキーのアテンションから算出される重みでバリューを集約したものです。したがって、文中の各要素に対してひとつのキー、バリューが対応しているはずです。一般的に、バリューはキーと同一になることもあります。

これは数学的にどういうことか見てみましょう。アテンションレイヤーは2つのステップからなります。（１）**アテンション機構**によるアテンションベクトル $\vec{b}$ の計算（２）アテンションベクトル $\vec{b}$ を用いたバリューの集約アテンション機構はアテンション方程式の別名です。上の例について考えてみてください。ここでは3次元の埋め込みを使って単語を表現してみます。

```{margin}
キーとクエリーがone-hotエンコーディングで与えられると、キーとクエリーが一致していない限り内積がゼロになるのでアテンションレイヤーの入力としては機能しない。
```


| インデックス| 埋め込み| 単語|
|:-----|:--------:|----:|
| 0    |  0,0,0   | The |
| 1    |  2,0,1   | Sleepy |
| 2    |  1,-1,-2   | Child |
| 3    |  2,3,1   | Reads |
| 4    |  -2,0,0   | A |
| 5    |  0,2,1   | Book |

キーはこれら全てをまとめたランク2のテンソル（行列）になります。ここでは、分かりやすさのために整数のみで説明していることに注意してください。典型的には単語の埋め込み表現には浮動小数点が用いられます。

\begin{equation}
\mathbf{K} = \left[
\begin{array}{lccccr}
0 & 2 & 1 & 2 & -2 & 0\\
0 & 0 & -1 & 3 & 0 & 2\\
0 & 1 & -2 & 1 & 0 & 1\\
\end{array}\right]
\end{equation}

この分には6つの単語があり、それぞれ3次元ベクトルで表現されるので、キーは $(6, 3)$ の形状をしています。
バリューは単純に、各単語に1つの値を持つとしましょう。これらのバリューによって出力を決定します。
もしかしたら、単語の感情を表現しているかもしれません：「happy」のようなポジティブな単語なのか、「angry」のようなネガティブな単語なのかなど。

\begin{equation}
\mathbf{V} = \left[ 0, -0.2, 0.3, 0.4, 0, 0.1\right]
\end{equation}

バリュー $\mathbf{V}$ はキーと同じランクであるべきで、その形状は $(6, 1)$になります。また、クエリーはキーよりランクが1小さくなります。この例でのクエリーは「book」という単語です。

\begin{equation}
\vec{q} = \left[0, 2, 1\right]
\end{equation}

## アテンション機構方程式

アテンションメカニズム方程式はクエリーとキーの引数のみを使用します。
この式はキーより1ランク低いテンソルを出力し、各キーに大してクエリーが持つべきアテンションに対応するスカラーを与えます。
このアテンションベクトルは正規化されている必要があります。最も一般的なアテンション機構は内積とソフトマックスです。

\begin{equation}
\vec{b} = \mathrm{softmax}\left(\vec{q}\cdot \mathbf{K}\right) = \mathrm{softmax}\left(\sum_j q_j k_{ij}\right)
\end{equation}

ここで、インデックス $i$ は系列における位置、$j$ は特徴量のインデックスをです。ソフトマックスは以下で定義され、

```{math}
:label: softmax
\mathrm{softmax}\left(\vec{x}\right) = \frac{e^\vec{x}}{\sum_i e^ x_i}
```

$\vec{b}$ が正規化されることを保証しています。上の例から得た値を代入すると、次のようになります。

\begin{equation}
\vec{b} = \mathrm{softmax}\left(\left[0, 2, 1\right] \times
\left[
\begin{array}{lccccr}
0 & 2 & 1 & 2 & -2 & 0\\
0 & 0 & -1 & 3 & 0 & 2\\
0 & 1 & -2 & 1 & 0 & 1\\
\end{array}\right]\right) = \mathrm{softmax}\left( \left[0, 1, -4, 7, 0, 5\right]\right)
\end{equation}

\begin{equation}
\vec{b}  = \left[0, 0, 0, 0.88, 0, 0.12\right]
\end{equation}

ここでは数字を丸めましたが、アテンションベクトルは単語自身（book）と動詞（read）にのみ重みを持っています。これは私が作った例ですが、アテンションが単語同士をどのように関連付けるか示唆を与えてくれています。グラフニューラルネットワークにおける近傍の概念を思い起こすかもしれません。

## アテンション集約

アテンションベクトル $\vec{b}$ は、バリューの加重平均の計算に使用されます。

\begin{equation}
 \mathbf{V}\vec{b} = \left[0, 0, 0, 0.88, 0, 0.12\right]^ T \left[ 0, -0.2, 0.3, 0.4, 0, 0.1\right] = 0.36
\end{equation}

概念的には、今回の例では文中クエリー「book」のアテンションで重みづけされた感情を計算したことになります。アテンションレイヤーは2つのことを行っていることが分かります：アテンション機構でアテンションベクトルを計算し、それを使ってバリューの加重平均を求めています。

## テンソルドット

この内積、ソフトマックス、集約はテンソルドットと呼ばれ、最も一般的なアテンションレイヤーです{cite}`luong2015effective`。一般的な派生としては、キーの次元（最後の軸の次元）で割ったものがあります。ここで、キーが正規化されていないことを思い出してください。乱数であれば、中心極限定理より内積からの出力の大きさははキーの次元の平方根でスケールします。つまり、$e^{\vec{q} \cdot \mathbf{K}}$ を取ることで、ソフトマックス値に悪影響を与える可能性があります。以上をまとめると、以下の式のようになります。

\begin{equation}
    \vec{b} = \mathrm{softmax}\left(\frac{1}{\sqrt{d}}\vec{q}\cdot \mathbf{K}\right)
\end{equation}

ここで、$d$ はクエリーベクトルの次元です。



## ソフト、ハード、温度アテンション

アテンションの派生形として考えられるのは、$\mathrm{softmax}$ の出力において最もアテンションの高いものを1に、それ以外を0に置き換えることです。これを**ハードアテンション**と呼びます。ハードアテンションの式は、以下のようにソフトマックスをハードマックスで置き換えることで定義されます。


\begin{equation}
\mathrm{hardmax}\left(\vec{x}\right) = \lim_{T\rightarrow0}\frac{e^\vec{x} / T}{\sum_i e^ {x_i / T}}
\end{equation}

これは、 $\vec{x}$ の最大要素の位置を1とし、それ以外の位置に0を置くことを数学的に定式化したものです。この式が統計力学のボルツマン分布に似ていることから、温度 $T$という用語を用いています。$T = 0$ のときはハードアテンション、$T = 1$ のときはソフトアテンション、$T = \infty$ のときは均一なアテンションを意味することがわかると思います。$T$ を中間的な値にすることも可能です。


## セルフアテンション

ディープラーニングでは、すべてがバッチ処理されることを覚えていますか？通常、アテンションレイヤーへのバッチ入力はクエリーです。これまでの議論では、クエリーはキーよりも1ランク低いテンソル（クエリー**ベクトル**）でしたが、バッチ化されるとキーと同じランクになります。ほとんどの場合、クエリーとキーは同じものです。我々の例では、クエリーは「book」の埋め込みベクトルで、これはキーのひとつでした。もし全ての単語を考慮するようにクエリーをバッチ処理すると、クエリーはキーと同じものになります。さらに特殊なケースとして、クエリー、バリュー、キー全てが同じ場合があり、これを**セルフアテンション**と呼びます。これはアテンションメカニズムがバリューを直接使用することを意味しており、レイヤーに入力される余分な「キー」は存在しません。

## 学習可能アテンション
ここまで説明してきたアテンションには、学習可能なパラメータは在りませんでした。アテンションによる学習はどのようにして行うのでしょうか？一般的に、学習可能なパラメータを直接式に持たせることはしません。代わりに、全結合層を通してキー、バリュー、クエリー（{doc}`layers`を参照）をアテンションへ入力します。その為、ひとつのレイヤーとしてアテンションを見た場合、学習可能なパラメータはありません。全結合層とアテンションレイヤーのブロックとしてみれば学習可能です。以下で明示的に確認しましょう。


## マルチヘッドアテンションブロック
複数のフィルターによる畳み込みに着想を得た、複数の並列アテンションからなるブロック（レイヤーのグループ）がある。
これらは「マルチヘッドアテンション」と呼ばれます。
もしバリューの形状が $(L, V)$ であれば、$(H, V)$ の形状のテンソルが返ってきます。ここで、$H$ は並列アテンションレイヤー（ヘッド）の数です。
アテンションレイヤーに学習可能パラメータがないのなら、どんな意味があるのでしょうか。
さて、重みを導入しましょう。全てのアテンションヘッドの形状が一定である必要があるので、重みは**正方**行列になっています。

アテンションレイヤーが $A(\vec{q}, \mathbf{K}, \mathbf{V})$ で定義されているとします。この時マルチヘッドアテンションは以下のように書けます。

\begin{equation}
\left[A(\mathbf{W}_q^0\vec{q}, \mathbf{W}_k^0\mathbf{K}, \mathbf{W}_v^0\mathbf{V}), A(\mathbf{W}_q^1\vec{q}, \mathbf{W}_k^1\mathbf{K}, \mathbf{W}_v^1\mathbf{V}), \ldots, A(\mathbf{W}_q^H\vec{q}, \mathbf{W}_k^H\mathbf{K}, \mathbf{W}_v^H\mathbf{V})\right]
\end{equation}

ここで、出力ベクトル $\ldots$ の各要素はアテンションレイヤーからの出力で、 $H$ 個の $(L, V)$ の形状をしたテンソルです。つまり、全体の出力としては $(H, L, V)$ の形状をしたテンソルになります。マルチヘッドアテンションブロックの最も有名な例は、トランスフォーマー{cite}`vaswani2018attention`で使用されているセルフアテンションマルチヘッドアテンションブロックです。通常、複数の連続したアテンションブロックを適用するので、次のブロックへ入力されるバリューは、ランク3 $(H, L, V)$ ではなくランク2のテンソルである必要があります。したがって、マルチヘッドアテンションの出力はしばしば $(H, V, V)$ または $(H)$ の重みテンソルとの行列積によってランク2になります。これが分かりにくいようであれば、以下の例を参照してください。

## ノートブックを実行する

上の &nbsp;<i aria-label="Launch interactive content" class="fas fa-rocket"></i>&nbsp; をクリックして、Google Colab を立ち上げてください。

## コードの例

アテンションがどのように実装されているか見てみましょう。ここでは様々な量にランダムな変数を使用するので、学習される変数を `w_` で、入力変数を `i_` で表すことにします。

### テンソルドット機構

まず、テンソルドット機構の実装から始めます。例として、系列の長さを11、キー特徴量の長さを4、バリュー特徴量の次元を2とします。キーとクエリーは、特徴量の次元が同じであることに注意してください。

In [ ]:
import numpy as np


def softmax(x, axis=None):
    return np.exp(x) / np.sum(np.exp(x), axis=axis)


def tensor_dot(q, k):
    b = softmax((k @ q) / np.sqrt(q.shape[0]))
    return b


i_query = np.random.normal(size=(4,))
i_keys = np.random.normal(size=(11, 4))

b = tensor_dot(i_query, i_keys)
print("b = ", b)

期待通り、合計が1のベクトル $\vec{b}$ が得られました。

### 一般的なAttention

では、このアテンション機構をアテンションレイヤーに組み込みましょう。

In [ ]:
def attention_layer(q, k, v):
    b = tensor_dot(q, k)
    return b @ v


i_values = np.random.normal(size=(11, 2))
attention_layer(i_query, i_keys, i_values)

各特徴次元に1つずつ、2つの値が得られます。

### セルフアテンション

セルフアテンションの変更点は、クエリー、キー、バリューを等しくすることです。この設定ではクエリーのバッチ処理が必要になり、ランク2の出力を得ます。

In [ ]:
def batched_tensor_dot(q, k):
    # a は batch x seq x feature 次元（ここでは N x N x 4）になる
    # アインシュタイン記法によるバッチ化されたドット積
    a = np.einsum("ij,kj->ik", q, k) / np.sqrt(q.shape[0])
    # 系列に対してソフトマックスを適用
    b = softmax(a, axis=1)
    return b


def self_attention(x):
    b = batched_tensor_dot(x, x)
    return b @ x


i_batched_query = np.random.normal(size=(11, 4))
self_attention(i_batched_query)

$11\times4$ の行列が得られれば上手く計算ができています。

### 学習可能パラメータを追加する

これらのステップに重み行列を追加することで、学習可能パラメータを追加することができます。セルフアテンションで実践してみましょう。セルフアテンションではキー、バリュー、クエリーは同じものでしたが、それぞれに異なる重みを掛けることができます。デモとして、バリューの特徴量次元を2に変更してみます。

In [ ]:
# 重みを入力次元 -> 所望の特徴量次元に変更する。
w_q = np.random.normal(size=(4, 4))
w_k = np.random.normal(size=(4, 4))
w_v = np.random.normal(size=(4, 2))


def trainable_self_attention(x, w_q, w_k, w_v):
    q = x @ w_q
    k = x @ w_k
    v = x @ w_v
    b = batched_tensor_dot(q, k)
    return b @ v


trainable_self_attention(i_batched_query, w_q, w_k, w_v)

重みでバリューの特徴量次元を2にしたので、 $11\times 2$の出力が得られます。

### マルチヘッド

マルチヘッドアテンションの唯一の変更点は各ヘッドに対して1つの重みを持ち、ヘッド適用後の出力を結合することです。学習可能な長さ $H$ の重みベクトルを使って出力を連結したり、平均や最大値などのリダクションを行います。

In [ ]:
w_q_h1 = np.random.normal(size=(4, 4))
w_k_h1 = np.random.normal(size=(4, 4))
w_v_h1 = np.random.normal(size=(4, 2))
w_q_h2 = np.random.normal(size=(4, 4))
w_k_h2 = np.random.normal(size=(4, 4))
w_v_h2 = np.random.normal(size=(4, 2))
w_h = np.random.normal(size=2)


def multihead_attention(x, w_q_h1, w_k_h1, w_v_h1, w_q_h2, w_k_h2, w_v_h2):
    h1_out = trainable_self_attention(x, w_q_h1, w_k_h1, w_v_h1)
    h2_out = trainable_self_attention(x, w_q_h2, w_k_h2, w_v_h2)
    # join along last axis so we can use dot.
    all_h = np.stack((h1_out, h2_out), -1)
    return all_h @ w_h


multihead_attention(i_batched_query, w_q_h1, w_k_h1, w_v_h1, w_q_h2, w_k_h2, w_v_h2)

期待通り、ランク2である $11\times 2$ の出力が得られました。

## グラフニューラルネットワークにおけるアテンション

グラフニューラルネットワークの重要な性質に permutation equivariant があることを思い出してください。
我々はグラフニューラルネットワークを permutation equivariant にするために、合計や平均などのリダクションを使用してきました。

また、アテンションレイヤーは permutation invariant（バッチ化しない時）もしくは permutation equivariant（バッチ化する時）です。このため、アテンションは近傍情報を集約する方法としてよく利用されています。アテンションレイヤーは重要な近傍を見つけるのが得意なので、高次元グラフ（大量な近傍を持つ）において重要です。これは分子では稀なことですが、全ての原子を結合してその距離をエッジとして置くだけで良いということです。グラフ畳み込みレイヤー（GCNレイヤー）やほとんどのGNNレイヤーが、レイヤーごとにひとつの結合しか情報を伝播させることができないことを思い出してください。したがって、全ての原子を結合してアテンションを適用することは、多数のレイヤーを経由しなくても長距離の情報伝達が可能になります。ただし、ネットワークが正しい結合/原子に注意を向けているか気を付ける必要があります。


アテンションが Battaglia equations{cite}`battaglia2018relational` にどう当てはまるか見てみましょう。Battaglia 方程式はGNNを定義するための一般的な標準方程式であることを思い出してください。アテンションは複数の場所に現れることがありますが、先述した通り近傍を考慮する時に現れます。具体的には、クエリーは $i$ 番目のノードとなり、キー／バリューは近傍ノードとエッジの特徴の組み合わせになります。Battaglia 方程式がきれいに当てはまるステップはないが、以下のようにアテンションレイヤーを分割することができます。アテンションレイヤーの大部分はエッジ更新式に当てはまるでしょう。


\begin{equation}
\vec{e}^{'}_k = \phi^e\left( \vec{e}_k, \vec{v}_{rk}, \vec{v}_{sk}, \vec{u}\right)
\end{equation}

これは一般化された式であり、 $\phi^e()$ の選択がGNNを定義していることを思い出してください。$\vec{e}_k$ はエッジ $k$ の特徴量ベクトル、$\vec{v}_{rk}$ はエッジ $k$ の受信ノード特徴量ベクトル、$\vec{v}_{sk}$ はエッジ $k$ の送信ノード特徴量ベクトル、$\vec{u}$ は全体グラフ特徴量です。このステップを、アテンション機構に利用します。ここで、クエリはー受信ノード $\vec{c}_{rk}$ で、キー／バリューは送信とエッジベクトルです。具体的には、Zhangらのアプローチ（{cite}`zhang2018gaan`）をテンソルドット機構で利用します。彼らはノード特徴量のみを考慮し、キーとバリューはノード特徴量と同一に設定しました。一方で、彼らはノード特徴量をキー／クエリーに変換する学習可能なパラメータを使用しました。


\begin{equation}
\vec{q} = \mathbf{W}_q\vec{v}_{rk}
\end{equation}

\begin{equation}
\mathbf{K} = \mathbf{W}_k\vec{v}_{sk}
\end{equation}

\begin{equation}
\mathbf{V} = \mathbf{W}_v\vec{v}_{sk}
\end{equation}

\begin{equation}
    \vec{b}_k = \mathrm{softmax}\left(\frac{1}{\sqrt{d}} \vec{q}\cdot \mathbf{K}\right)
\end{equation}


\begin{equation}
\vec{e}^{'}_k = \vec{b} V
\end{equation}

ひとつの式にまとめると：

\begin{equation}
\vec{e}^{'}_k = \mathrm{softmax}\left(\frac{1}{\sqrt{d}} \mathbf{W}_q\vec{v}_{rk}\cdot \mathbf{W}_k\vec{v}_{sk}\right)\mathbf{W}_v\vec{v}_{sk}
\end{equation}

これで、アテンションから重みづけされたエッジ特徴量ベクトルを得ることができます。
最後に、エッジ集約ステップでこれらのエッジ特徴量を合計します。

\begin{equation}
\bar{e}^{'}_i = \rho^{e\rightarrow v}\left( E_i^{'}\right) = \sum E_i^{'}
\end{equation}

Zhangら{cite}`zhang2018gaan`では、マルチヘッドアテンションも使用していました。
マルチヘッドアテンションはどのように機能するのでしょうか？

エッジ特徴量行列 $E_i^{'}$ は軸0がエッジ（$k$）、軸1が特徴量、軸2がヘッドのエッジ特徴量テンソルになります。「ヘッド」は単に $\mathbf{W}^h_q, \mathbf{W}^h_k, \mathbf{W}^h_v$ のどの集合を使ったかを意味していることを思い出してください。
テンソルを期待される行列に戻すためには、単純に最後の2軸（特徴量、ヘッド）を特徴量にマップする重み行列を用いれば良いです。

わかりやすさのため、インデックスを明示的に書き出しましょう：

\begin{equation}
\bar{e}^{'}_{il} = \rho^{e\rightarrow v}\left( E_i^{'}\right) = \sum_k e_{ikjh}^{'}w_{jhl}
\end{equation}

ここで、 $j$ はエッジ特徴量の入力インデックス、＄l$ は出力エッジ特徴量行列で、$k,h,i$ は以前と同様の定義とします。**トランスフォーマー**はマルチヘッドアテンションで構築されたネットワークの別名なので、トランスフォーマーグラフニューラルネットワークも見かけることがあるでしょう（{cite}`maziarka2020molecule`）。

## 章のまとめ

* アテンションレイヤーは人間の注意機構にヒントを得ているが、基本的には加重平均によるリダクションである。
* アテンションレイヤーはクエリー、バリュー、キーの3つの入力をとる。これらの入力はしばしば同一で、クエリはキーの１つであり、キーとバリューは等しい。
* 言語のような系列モデリングに向いている。
* アテンションベクトルは正規化されている必要がある。ソフトマックス関数で正規化を実現できるが、アテンション機構式はハイパーパラメータである。
* アテンションレイヤーはアテンション機構でアテンションベクトルを計算する、そしてアテンション加重平均を計算することでアテンションベクトルのリダクションを行う。
* ハードアテンション（ハードマックス関数）を用いると、アテンション機構の最大出力を返す。
* ソフトマックス後のテンソルドットはアテンション機構でもっとも一般的である。
* セルフアテンションはクエリー、バリュー、キーが全て等しいときに達成される。
* アテンションレイヤーそのものは学習できない。
* マルチヘッドアテンションブロックは複数並列アテンションに分割可能なレイヤーのグループである。

## 引用文献

```{bibliography}
:style: unsrtalpha
:filter: docname in docnames
```